# Análise e Histórico de Logs de uso da Aplicação

Este *notebook* tem como objetivo permitir a análise dos *logs* de utilização da aplicação, disponível via CLI (*Command Line Interface*). Em uma situação de uso real, os *logs* seriam registrados em uma base de dados ou ferramenta mais adequada (e.g., Elasticsearch, MongoDB e DataDog). Para este projeto, os dados são registrados em arquivos e permitem posterior recuperação.

## Bibliotecas e Funções

In [1]:
# General
import sys
from typing import List
from pathlib import Path

# Visualization / Presentation
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML, display

import json
import pandas as pd

# Carregar, além de atualizar frequentemente, código personalizado disponível em ../src
%load_ext autoreload 
%autoreload 2
sys.path.append(str(Path.cwd().parent))
from src import settings
from src.utils.notebooks import display_side_by_side

# Configurações para a exibição de conteúdo do Pandas e das bibliotecas gráficas
%matplotlib inline 
sns.set(rc={'figure.figsize':(25,10)})
pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option('max_colwidth', 150)


def retrieve_log(level: str) -> List[dict]:
    file_path = Path(settings.APPLICATION_LOG_PATH).joinpath(f'{level}.log')
    json_items = []
    with open(file_path, 'r') as file:
        json_items = [json.loads(item)
                      for item in file.readlines()]
        
    return json_items

## Recuperação dos *Logs* de Resultados de Ações

Apesar de serem registrados *logs* para informações gerais, erros e outras informações relevantes, um dos principais históricos de interesse é o de utilização da aplicação para acompanhar os resultados gerados pelas interações. Recordando, são 3 interações disponíveis via CLI:
 - Classificação da categoria de produto;
 - Classificação da intenção de busca;
 - Recomendação de produtos a partir de busca.

Como os modelos de classificação têm quantidade de menor de decisões e parâmetros envolvidos com relação ao modelo de recomendação, há diferenças nas informações que são registradas nos logs. A exemplo disso, há duas estruturas de *logs* de modelos, uma simples e uma composta. Ambas possum em comum dados do momento de execução, módulo utilizado, tempo total de execução e identificação da versão utilizada do modelo.

In [2]:
logs_frame = pd.json_normalize([item
                                for item in retrieve_log('info')
                                if item['type'] == 'model_usage'])

simple_log = (logs_frame
              .loc[lambda f: f['action'].str.startswith('classify')]
              .head(1)
              .reset_index()
              .dropna(axis=1)
              .rename(index={0: 'Value'})
              .T
             )

compound_log = (logs_frame
                .loc[lambda f: f['action'].str.startswith('recommend')]
                .head(1)
                .dropna(axis=1)
                .rename(index={0: 'Value'})
                .T
               )

display_side_by_side([simple_log, compound_log], 
                     ['Dados de Modelo Simples', 'Dados de Modelo Composto'])

del simple_log, compound_log

,Value
index,0
written_at,2021-06-11T16:00:25.535Z
written_ts,1623427225535852000
msg,Classified Products Categories
type,model_usage
logger,src.logging
thread,MainThread
level,INFO
module,inference_pipeline
line_no,75


A seguir, os registros de resultados são recuperados e os dados de execuções e tempo de execução são sumarizados.

In [3]:
agg_frame = (logs_frame
             [['action', 'input_length', 'total_time']]
             .assign(executions=1)
             .groupby(['action'])
             .agg({'executions': ['sum'],
                   'total_time': ['sum', 'mean', 'std', 'min', 'max']})
            )

display_side_by_side([agg_frame],
                     ['Resumo dos Logs de Utilização dos Modelos'])
del agg_frame

Como os dados evidenciam, os modelos de classificação tendem a ter quase 1/4 do tempo médio de execução da recomendação. Isso ocorre, porque a recomendação utiliza os dois modelos de classificação (de categoria e de intenção) e pelo fato de existir a necessidade de se recuperar dados em disco e fazer operações de cálculos, filtro e ordenação sobre eles.

## Análise de Recomendações

Como o modelo de recomendação é o mais complexo construído neste projeto, envolvendo o uso de outros 2 classificadores e com critérios adicionais de pontuação dos dados, algumas informações adicionais serão analisadas.

In [4]:
prob_columns = [c for c in logs_frame.columns if 'categories_probs' in c]

recs_columns = (['query', 'query_intent', 'selected_categories', 'products_dataset_length', 'category_filtered_products_dataset_length', 'recommendations_length'] + 
                prob_columns + 
                ['total_time']                
               )

recs_frame = (logs_frame
              .loc[lambda f: f['action'].str.startswith('recommend')]
              [recs_columns]
             )

for column in prob_columns:
    recs_frame[column] = recs_frame[column].apply(lambda v: f'{v * 100:.2f}%')

(recs_frame
 .rename(columns={c:c.replace('categories_probs.', 'Prob_') for c in prob_columns}) 
)

,query,query_intent,selected_categories,products_dataset_length,category_filtered_products_dataset_length,recommendations_length,Prob_Bebê,Prob_Bijuterias e Jóias,Prob_Decoração,Prob_Lembrancinhas,Prob_Outros,Prob_Papel e Cia,total_time
13,presente de casamento,Exploração,"[Lembrancinhas, Decoração]",25156.0,17127.0,10.0,2.25%,0.01%,36.42%,51.90%,1.80%,7.63%,11.480723
15,lembrancinha de chá de bebê,Exploração,[Lembrancinhas],25156.0,10827.0,10.0,24.83%,0.00%,0.03%,75.09%,0.01%,0.05%,11.421238
17,papel fotográfico glossy folha A4,Foco,[Papel e Cia],25156.0,1755.0,10.0,0.03%,0.00%,0.76%,0.19%,0.00%,99.01%,11.553423


Mesmo com uma quantidade limitada de utilizações da aplicação, é possível aproveitar os registros do *log* para verificar o comportamento correto dos modelos e verificar as diferenças práticas da lógica implementada. Nos dados apresentados, é possível notar que as consultas classificadas com intenção de `exploração` têm pelo menos uma ordem de magnitude a mais do que a intenção de `foco`. Em parte, isso ocorre pelo fato de a busca focada não envolver a categoria com maior quantidade de registros (*Lembrancinhas*), mas também pelo fato de a intenção permitir apenas a recuperação de resultados da categoria principal do produto.

Os registros também mostram o funcionamento esperado do modelo, para a intenção de `exploração`, de adotar um limiar mínimo de uso de categorias. Por esse motivo, há um exemplo de consulta exploratória com duas categorias, indicadas pelo modelo com probabilidade maior do que 25%, e uma consulta com categoria única, que possui um perfil mais claro para o modelo, com 99% de probabilidade.

Apesar das diferenças entre as intenções e as categorias das consultas, não há uma diferença relevante no tempo de execução do modelo. Além disso, por não existir um limiar de corte da pontuação dos produtos, o total de produtos recomendados não é afetado. Isso, no futuro, pode ser alterado com um estudo do critério de corte.

## Resulados de Recomendações

Além de poder analisar o comportamento geral das recomendações e os critérios adotados, pode-se olhar também os resultados em mais detalhes. A exemplo disso, os resultados da busca `presente de casamento` serão exibidos.

In [42]:
products_frame = (pd
                   .DataFrame(logs_frame
                              .loc[lambda f: f['query'] == 'presente de casamento']
                              .iloc[0]
                              ['recommendations']
                             )
                  )

display_side_by_side([products_frame],
                     ['Critérios de Pontuação de Produtos para Recomendação'])

,product_id,title,score,title_similarity,concatenated_tags_similarity,category_prob,orders_per_views
0,12705294,LEMBRANCINHA DE CASAMENTO,1.651256,0.831071,0.735258,0.518963,0.037037
1,12181007,LEMBRANCINHA DE CASAMENTO,1.649572,0.831071,0.735258,0.518963,0.030303
2,9793031,Lembrancinha de casamento,1.645602,0.831071,0.735258,0.518963,0.014423
3,6721352,lembrancinha de casamento,1.644548,0.831071,0.735258,0.518963,0.010204
4,7986446,Lembrancinha de Casamento,1.643047,0.831071,0.735258,0.518963,0.004202
5,743898,Lembrancinha de Casamento,1.628882,0.831071,0.713553,0.518963,0.012658
6,1334030,Lembrança de casamento,1.628603,0.846299,0.693175,0.518963,0.011765
7,16488265,Lembrancinha de Casamento,1.626543,0.831071,0.713553,0.518963,0.003300
8,14835882,lembrancinhas de casamento,1.618163,0.806064,0.735258,0.518963,0.004695
9,16400938,lembrancinhas de casamento,1.617909,0.806064,0.735258,0.518963,0.003676


Pelos resultados, é possível notar que a diferença do uso de letras maiúsculas ou minúsculas, entre os 6 primeiros itens, não influenciou a pontuação de similaridade dos títulos. Como todos os itens pertencem à mesma categoria (*Lembrancinhas*), os critérios que têm influência sobre a pontuação final são *tags concatenadas* e *pedidos por visualizações*. 

Algo que pode ter a atenção chamada é a importância do uso de *pedidos por visualizações*, que permite fazer a diferenciação de critérios entre os produtos com características semelhantes. Por esse critério, produtos que tenham maior quantidade de pedidos por visualização, que serve como uma medida de sucesso de venda, são exibidos primeiro. Cabe ressaltar que essa taxa tem um teto, o que evita que ela perpetude a dominância de um produto sobre outros.